$$
\kappa(u) = \left(\frac{3u(1-u)}{3u^3 + (1-u)^3}\right)^2  + \mu
$$
$$
\begin{equation}
\left\{
  \begin{aligned}
  &-\nabla\cdot(\kappa(u)\nabla u) = f, &x \in \Omega, \\
  &u = 0, &x \in \partial\Omega
\end{aligned}
\right.
\end{equation}
$$

$$
  f(x; i^\star) = 100 \exp(-50 ||x - x_{c, i^\star}||_2^2)\mathbf{1}_{\Omega_{i^\star}}(x)
$$

In [ ]:
from utils import fd_solve_nlinear
import numpy as np

GridSize = 64
area = ((0, 0), (1, 1))
mu = 0.1

ans = []
for i in np.arange(0.1, 1, 0.1):
    for j in np.arange(0.1, 1, 0.1):
        center = (i, j)
        ans.append(fd_solve_nlinear(GridSize, area, mu, center,))

np.stack(ans)
np.save(f'./DLdata/nlinear/{GridSize}/U.npy', ans)

In [ ]:
from FVM.src.UniformICD import UniformFVM
from FVM.src.Problem.test_uniform import *
from FVM.src.utils import show_errors

# Test zxp case !!!
case_ns = list(range(10))
print(f"|{'Method': ^8s}|{'Case ID': ^8s}|{'Mesh Type': ^18s}|{'hmesh': ^11s}|{'error_max': ^11s}|{'ratio': ^7s}|{'error_l2': ^11s}|{'ratio': ^7s}|{'error_h1': ^11s}|{'ratio': ^7s}|")
print(f'|{":--:": ^8s}|{":--:": ^8s}|{":--:": ^18s}|{":--:": ^11s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|')
for case_n in case_ns:
    mesh_type = 'quadrangle_uniform'
        # triangle_classic triangle_random triangle_uniform triangle_kershaw
        # quadrangle_uniform quadrangle_random  quadrangle_kershaw
    n_levels = 5
    hmeshes = []
    errors_max = []
    errors_l2 = []
    errors_h1 = []

    for i in range(n_levels):
        N = 2**(i+3)
        problem = zxpProblem(N, N, ((0, 0), (1, 1)), case_n, 1e-9)
        solver = UniformFVM(((0, 0), (1, 1)), N, N, problem)
        solver.solve(solver_name=None)
        hmesh, error_max, error_l2, error_h1 = solver.compute_errors()

        hmeshes.append(hmesh)
        errors_max.append(error_max)
        errors_l2.append(error_l2)
        errors_h1.append(error_h1)

    for s in show_errors(hmeshes, errors_max, errors_l2, errors_h1):
        print(f'|{"ICD": ^8s}|{case_n: ^8}|{mesh_type: ^18s}|' + s)

In [ ]:
from Problems.NonlinearProblem import *
from plots import my_plot
from UniformICD import UniformFVM

def fvm_solve_nlinear(GridSize, area, mu, center=(0.5, 0.5), 
                    Picard_maxiter=1000, eps=1e-7):
    u0 = np.random.rand(GridSize, GridSize)
    problem = NonLinearProblem(None, center, N, area)
    solver = UniformFVM(area, N, N, problem)
    solver.solve(solver_name = None)
    u0,  b = solver.ua, solver.b

    for i in range(Picard_maxiter):
        problem = NonLinearProblem(kappa(u0.reshape(GridSize, GridSize), mu), center, N, area)

        solver = UniformFVM(area, N, N, problem)
        solver.solve(solver_name = None)
        newA = solver.A.tocsr()
        newu = solver.ua

        # delta = ((newu - u0)**2 * h**2).sum()
        delta = np.linalg.norm(newu - u0)
        # error = ((newA @ u0 - b)**2 * h**2).sum()
        error = np.linalg.norm(newA @ u0 - b)
        print(f"Itr: {i}\t Delta: {delta:.3e}\t Error: {error:.3e}\t")

        if delta <= eps or error <= eps:
            u = newu.reshape(GridSize, GridSize)
            break
        else:
            u0, A0 = newu, newA
    return u


N = 128
mu = 0.1
center = (0.1, 0.1)
area = ((0, 0), (1, 1))

solution = fvm_solve_nlinear(N, area, mu)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Problem setup
Lx, Ly = 1.0, 1.0  # Domain size
Nx, Ny = 200, 200  # Number of points in each direction
dx, dy = Lx / (Nx - 1), Ly / (Ny - 1)  # Grid spacing
u0, u1, u2, u3 = 0.0, 1.0, 0.0, 1.0  # Boundary conditions

# Initialize the u array
u = np.zeros((Nx, Ny)) + 0.5  # Initial guess

# Apply boundary conditions
u[0, :] = u0  # Left
u[-1, :] = u1  # Right
u[:, 0] = u2  # Bottom
u[:, -1] = u3  # Top

# Function to compute kappa(u)
def kappa(u):
    return ((3 * u * (1 - u)) / (3 * u**3 + (1 - u)**3))**2 + 0.5

# Iteration parameters
max_iters = 1000
tolerance = 1e-6
error = 1
iters = 0

# Iterative solver (simple fixed-point iteration)
while error > tolerance and iters < max_iters:
    u_old = u.copy()
    print(f"Itr: {iters}\t Error: {error:.3e}\t")
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            # Compute kappa values at half points
            kappa_e = kappa((u[i+1, j] + u[i, j]) / 2)
            kappa_w = kappa((u[i, j] + u[i-1, j]) / 2)
            kappa_n = kappa((u[i, j+1] + u[i, j]) / 2)
            kappa_s = kappa((u[i, j] + u[i, j-1]) / 2)
            
            # Update u based on discretization
            u[i, j] = ((kappa_e * u[i+1, j] + kappa_w * u[i-1, j]) / dx**2 +
                       (kappa_n * u[i, j+1] + kappa_s * u[i, j-1]) / dy**2) / \
                      ((kappa_e + kappa_w) / dx**2 + (kappa_n + kappa_s) / dy**2)
    
    # Compute error
    error = np.max(np.abs(u - u_old))
    iters += 1

# Plotting the solution
X, Y = np.meshgrid(np.linspace(0, Lx, Nx), np.linspace(0, Ly, Ny))
plt.contourf(X, Y, u.T, levels=50, cmap='viridis')
plt.colorbar()
plt.title('Steady-State Nonlinear Diffusion')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

iters, error


In [ ]:
from fit_multibc_heat import *
import numpy as np
import torch

layouts = np.load('DLdata/heat/GridSize-128/F.npy')
layouts = torch.from_numpy(layouts)
ds = C2DS(((0, 0), (0.1, 0.1)), 128, layouts, torch.float, 'cuda')

dl = DataLoader(ds, 3)
for data, layout, boundary, bd_case in dl:
    print(data.shape)
    print(layout.shape)
    print(boundary.shape)
    print(bd_case)
    for i, c in enumerate(bd_case):
        print(i, c.item())
        print(type(c.item()))
    break


In [1]:
from utils import coo2tensor

from Generators import CGTorch
from scipy.sparse import load_npz
from scipy.sparse.linalg import spsolve
from plots import fig_ax_for_save
import matplotlib.pyplot as plt
import numpy as np
import torch

N = 128
dtype = torch.double
device = 'cpu'

h = 0.1 / N
xx, yy = np.meshgrid(
    np.arange(h/2, 0.1, h),
    np.arange(h/2, 0.1, h)
)

A = load_npz('DLdata/heat/GridSize-128/case-0/A.npz').tocoo()
b = np.load('DLdata/heat/GridSize-128/case-0/b.npy')
F = np.load('DLdata/heat/GridSize-128/F.npy')[:5]
b = torch.from_numpy(b).to(dtype).to(device)
F = torch.from_numpy(F).to(dtype).to(device)
b = b + torch.flatten(F, 1, -1) * h**2

u = []
for i in range(5):
    u0 = spsolve(A.tocsr(), b[0].numpy())
    # u0 = u0.reshape(N, N)
    u.append(u0)

u = np.stack(u)
u = torch.from_numpy(u).to(dtype).to(device)
# fig, ax = fig_ax_for_save()
# ax.contourf(xx, yy, u0, levels=50)
A = coo2tensor(A, 'cpu', torch.float)
generator = CGTorch(A, dtype, device)

vecs = torch.rand(5, N, N, dtype=dtype)
# sol = generator(u.reshape(5, N, N), b, 1000)
sol = generator(vecs, b, 1000)


error: 6.199e+00
error: 4.013e+00
error: 3.145e+00
error: 2.432e+00
error: 2.065e+00
error: 1.736e+00
error: 1.527e+00
error: 1.340e+00
error: 1.205e+00
error: 1.085e+00
error: 9.908e-01
error: 9.069e-01
error: 8.374e-01
error: 7.755e-01
error: 7.222e-01
error: 6.746e-01
error: 6.325e-01
error: 5.947e-01
error: 5.606e-01
error: 5.299e-01
error: 5.017e-01
error: 4.762e-01
error: 4.525e-01
error: 4.311e-01
error: 4.109e-01
error: 3.925e-01
error: 3.752e-01
error: 3.593e-01
error: 3.442e-01
error: 3.303e-01
error: 3.171e-01
error: 3.048e-01
error: 2.931e-01
error: 2.823e-01
error: 2.718e-01
error: 2.621e-01
error: 2.527e-01
error: 2.440e-01
error: 2.355e-01
error: 2.276e-01
error: 2.199e-01
error: 2.128e-01
error: 2.058e-01
error: 1.992e-01
error: 1.928e-01
error: 1.869e-01
error: 1.810e-01
error: 1.755e-01
error: 1.701e-01
error: 1.650e-01
error: 1.600e-01
error: 1.553e-01
error: 1.507e-01
error: 1.463e-01
error: 1.420e-01
error: 1.379e-01
error: 1.339e-01
error: 1.301e-01
error: 1.264e-